In [1]:
import findspark

In [2]:
findspark.init()

In [3]:
findspark.find()

'/usr/local/lib/python3.10/dist-packages/pyspark'

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("sql").master("local[4]").getOrCreate()
spark

23/09/02 15:44:20 WARN Utils: Your hostname, pupil-a resolves to a loopback address: 127.0.1.1; using 167.235.141.210 instead (on interface eth0)
23/09/02 15:44:20 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/02 15:44:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
death_df = spark.read.option("header", True).option("inferSchema", True).csv("deaths.csv")
death_df.printSchema()


root
 |-- country: string (nullable = true)
 |-- placename: string (nullable = true)
 |-- frequency: string (nullable = true)
 |-- start_date: date (nullable = true)
 |-- end_date: date (nullable = true)
 |-- year: string (nullable = true)
 |-- month: integer (nullable = true)
 |-- week: integer (nullable = true)
 |-- deaths: integer (nullable = true)
 |-- expected_deaths: integer (nullable = true)
 |-- excess_deaths: integer (nullable = true)
 |-- baseline: string (nullable = true)



In [6]:
death_view = death_df.createOrReplaceTempView("bla")


In [7]:
type(death_view)

NoneType

In [49]:
spark.sql("SELECT * FROM bla").show()

+-------+---------+---------+----------+----------+----+-----+----+------+---------------+-------------+--------------------+
|country|placename|frequency|start_date|  end_date|year|month|week|deaths|expected_deaths|excess_deaths|            baseline|
+-------+---------+---------+----------+----------+----+-----+----+------+---------------+-------------+--------------------+
|Austria|     null|   weekly|2020-01-06|2020-01-12|2020|    1|   2|  1702|           1806|         -104|2015-2019 histori...|
|Austria|     null|   weekly|2020-01-13|2020-01-19|2020|    1|   3|  1797|           1819|          -22|2015-2019 histori...|
|Austria|     null|   weekly|2020-01-20|2020-01-26|2020|    1|   4|  1779|           1831|          -52|2015-2019 histori...|
|Austria|     null|   weekly|2020-01-27|2020-02-02|2020|    2|   5|  1947|           1837|          110|2015-2019 histori...|
|Austria|     null|   weekly|2020-02-03|2020-02-09|2020|    2|   6|  1681|           1837|         -156|2015-2019 hist

In [9]:
import generate_test_data


hallo


In [10]:
generate_test_data.generate()

check
check


In [3]:
kunden = [
    {
        "id": "foo"
    },
    {
        "id": "bar"
    }
]

In [4]:
list(kunden[0].keys())

['id']

In [11]:
from names_generator import generate_name

In [13]:
generate_name(style="capital")

'Amazing Brown'

In [4]:
import time
from datetime import datetime

In [5]:
ts = time.time()

In [6]:
dt = datetime.fromtimestamp(ts)

In [7]:
dt

datetime.datetime(2023, 9, 3, 6, 45, 31, 53979)

In [8]:
kunden_df = spark.read.option("header", True).csv("kunden.csv")

NameError: name 'spark' is not defined

In [9]:
kunden_df.show()

NameError: name 'kunden_df' is not defined

In [9]:
start_ts = datetime.strptime('9/1/2023 1:30 PM', '%m/%d/%Y %I:%M %p')

In [10]:
end_ts = datetime.strptime('9/20/2023 1:30 PM', '%m/%d/%Y %I:%M %p')

In [12]:
end_ts

datetime.datetime(2023, 9, 20, 13, 30)

In [15]:
def random_between(start, end):
    """
    This function will return a random datetime between two datetime 
    objects.
    """
    from random import randrange
    from datetime import timedelta
    delta = end - start
    int_delta = (delta.days * 24 * 60 * 60) + delta.seconds
    random_second = randrange(int_delta)
    return start + timedelta(seconds=random_second)

In [16]:
random_between(start_ts, end_ts)

datetime.datetime(2023, 9, 15, 2, 17, 8)

In [1]:
import generate_test_data

In [2]:
generate_test_data.main(1000)

In [2]:
weighted_classes = {
    "mit_kunde": 10,
    "ohne_kunde": 8
}

In [41]:
type(weighted_classes)

dict

In [48]:
draw_item(weighted_classes)

'ohne_kunde'

In [9]:
generate_test_data.draw_item(weighted_classes)

'ohne_kunde'

In [1]:
import generate_test_data

In [2]:
generate_test_data.main(1000)